In [7]:
import sys
sys.path.append("..")
from lib.util import add_income_group, add_race_eth, add_purpose_type
import pandas as pd
import numpy as np
from ambry import library
l = library()
p = l.get('ffiec.gov-hmda-2010e-lar-2012-ca').partition
df = p.select("SELECT * FROM lar WHERE county_code = 73").pandas


In [2]:
# Add the race an ethnicity column, race_eth
add_race_eth(df)

dfs = df[['id','census_tract_number', 'applicant_ethnicity','applicant_race_1', 'race_eth']]
dfc = dfs.groupby(['census_tract_number', 'race_eth']).count()[['id']]

# Pivot to get the race_eth as columns
df_re = dfc.reset_index().pivot('census_tract_number', 'race_eth','id')
df_re.head()


race_eth,aian,as,b,h,pi,w
census_tract_number,,,,,,
0001.00,1,7,3,15,NaN,347
0002.01,NaN,7,NaN,12,2,189
0002.02,NaN,23,2,21,NaN,355
0003.00,2,15,3,18,1,282
0004.00,NaN,9,2,2,NaN,115


From : http://www.ffiec.gov/geocode/help3.aspx

Tract Income Level - This corresponds to tract classifications as defined by the HMDA and CRA regulations. This field is based on the Tract Median Family Income %:

If the Median Family Income % is < 50% and > 0 then the Income Level is Low.

If the Median Family Income % is >= 50% and < 80% then the Income Level is Moderate.

If the Median Family Income % is >= 80% and < 120% then the Income Level is Middle.

If the Median Family Income % is >=120% then the Income Level is Upper.

If the Median Family Income % is 0% then the Income Level is Not Known.

In [4]:
# Needs to be converted from a string, since some entries are not numbers
df.applicant_income = df.applicant_income.astype(float, raise_on_error=False)

# Add the 'income_group'
add_income_group(df)

In [5]:
    
dfi = df[['applicant_income','hud_median_family_income', 'tract_to_msa_md_income','income_group']]
dfi.income_group.value_counts()
    

low     207208
na       28465
mod       4278
mid       1557
high      1198
dtype: int64

In [9]:
msa = df[(df.msa_md  == 41740) & ( df.respondent_id == '0000480228' )].copy() # San Diego MSA / Bank of America
add_income_group(msa) # adds income_group
add_race_eth(msa) # adds race_eth
add_purpose_type(msa) # add purpose_type

In [18]:
msa.groupby(['purpose_type', 'action_type']).count()[['id']].unstack()

id                          
action_type               1    2     3    4    5    6
purpose_type                                         
improvement              52    1    35    6    6    3
purchase-conventional   622   43   204  115   13   25
purchase-govt           243  NaN   100   48    5  154
refinance              5439  214  1577  601  296  144

In [30]:
msa.groupby(['applicant_race_1', 'action_type']).count()[['id']].unstack()

id                          
action_type          1    2     3    4    5    6
applicant_race_1                                
1                   41    1    19    6    4    5
2                  662   27   206   88   36   17
3                  133    1    36   12   10    9
4                   51    4    16   10    5    6
5                 4365  167  1299  528  219  232
6                 1089   57   337  123   45   57
7                   15    1     3    3    1  NaN